## Project 2: Food Facts
#### Presented by Carlee Price, Yubo Zhang, and Nikki Haas
#### August 2015

### Introduction

We used Open Food Facts dataset for our final project.  The database is a "free, open and collaborative database of food products from the entire world". The database includes 150 fields (both numerical and descriptive) and over 90,000 items.  For this project, we will work with the subset of data which includes products available in the US (2,800 items).  We'll ask of this data questions related to nutritional content, and for that purpose will pull in data from additional sources including the IOM.

The data was groomed and manipulated using the Pandas, Numpy, Matplotlib, Regular Expressions and SciPy modules avaialbe for Python.

### Question 1: Are these foods nutritious?

The FDA recommends to Americans certain levels of daily nutrient intake, based on IOM research. Do the foods available in our supermarkets generally allow us to meet these recommended intake levels?  Are there certain nutrients that we must work harder to consume than others?  We start with the list of nutrients for which IOM RDAs are available<sup>1</sup>, and the full list of nutrients for which our dataset offers a field.

*include our IOM table here that shows nutrient recommendations*

*include a list of the nutrient fields available in our dataset*

#### *Step One: Exploratory Analysis*

#### *Finding: *  Insufficient information is available at the point of purchase, regarding the nutrient content of many of these products.

Many of the 150 data fields were sparesely populated in our subset. FDA labelling guidelines<sup>2</sup> only require reporting of select macronutrients, and leave most essential vitamins and minerals and all micronutrients under the umbrella of voluntary declaration<sup>3</sup>. Voluntary declarations are most commonly made on foods that present themselves as "healthier"<sup>4</sup>. We cannot conclude that foods that lack the declaration of these nutrients, lack the nutrients themselves.  Instead, we will adjust the list of nutrients we seek to study, to those for which our dataset contains a meaningful numer of entries, and will adjust our product list (2,800 items) to those that are well-described (900 items).  The new set, while smaller, was deemed sufficiently large to proceed.

*include table here showing longer list of nutrients and their counts.*

*include list (or table) here showing the nutrients left in our study*


#### *Step Two: Tidy remaining data and verify*

#### *Finding: *  Nutrition information is provided in a variety of ways and wants for standardization.

After structuring the data set to include just the products for which sufficient nutrient information was present, and narrowing our list of interesting nutrients likewise, we worked to validate the data.  The energy_100g field was particularly troubling; as this was to form the foundation for subsequent analysis, we chose to rebuild the field using data that we were more confident was reliable.  We also needed to verify that units were consistent among our data and between our working data sets (the product sample and the IOM table).   

*include something there that shows our process for rebuilding energy field*

*show raw data IOM table v product, and then groomed*

#### *Step Three: Build an Averge Daily Intake Profile*

#### *Finding: *  An average sampling of these products provides insufficient vitamins and minerals.

From the groomed data for the smaller set, we took a simple average on each remaining field.  This represents the average content of that nutrient, among the products in our set.  From this, we can build an average composite daily intake representation.  Starting with calorie consumption recommendations (2500 for an adult male), and using our new calories_100g field, we determined that a person could consume 862.7g of food from our sample set, each day. This amount of food provided quantities of nutrients that were below recommended levels on all counts.       

*include table that shows recommendations and measured amounts*

### Biases & Limitations:

We sized down our data set at several points.  Our final analysis was based on a 900 X 15 dataframe.  Had more of the fields been populated, or had our starting set been larger (French products for example in Open Food Facts number greater than 56,000) there may have been more room for nuanced analysis.  

The data set is built on information from each food's packaging (versus, for example, laboratory analysis).  As such it reflects and is limited by industry standards and regulations.  The US FDA requires reporting of only a few choice nutrients.  Voluntary reporting of non-statutory nutrients is limited. Our analysis, therefore, cannot be considered an examination of the true content and nutritional value of these foods, but rather the information that a consumer of the food could reasonable discern.  In all likelihood, a diet of these foods would be more robust and nutritionally satisfying than we can with certainty say here.

Likewise, a set of packaged foods will naturally deselect for some important food categories.  Fruits, vegetables, bulk foods (nuts, beans, grains) do not appear in our set<sup>5<\sup>.  The role of these products in an average consumer's diet and more importantly their nutrient intake, is categorically understated in this study.

### Question One Conclusion:

Packaged foods available in the US do not provide a robust source of declared nutrients.  Vitamin B is particularly deficient.

1. source: https://www.consumerlab.com/RDAs/
2: https://www.federalregister.gov/articles/2016/05/27/2016-11867/food-labeling-revision-of-the-nutrition-and-supplement-facts-labels#h-31
3. FDA considers required label inclusion for "non-statutory nutrients...for which there is an independent relationship between the nutrient and risk of chronic disease, health-related condition, or physiological endpoint."
4. http://world.openfoodfacts.org/product/0082592720153
5. Chart that includes "vegetable" type product categories & their counts

In [76]:
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt
import re
import html5lib

3.5.2 |Anaconda 4.1.1 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
1.11.1
0.18.1


In [77]:
# df will be our set with all fields, it originated by searching the foodfacts website for US products
# we're going to work with Nikki's csv, which includes her calories_100g field

df = pd.read_csv("groomed_food_facts_data.cs", sep = '\t')

# and also include her adjustment for the fields that were truncating

pd.set_option('display.max_colwidth', -1)
df.head()

,Unnamed: 0,code,url,creator,created_t,last_modified_t,product_name,generic_name,quantity,packaging,...,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,calories_100g
0,0,5010092093045,http://world.openfoodfacts.org/product/5010092093045,bcatelin,1389309305,1461479010,Soft white,White bread,800g,Plastic bag,...,NaN,NaN,NaN,NaN,NaN,NaN,125.0,-1.0,-1.0,232.400
1,1,44000030377,http://world.openfoodfacts.org/product/0044000030377,openfoodfacts-contributors,1385850411,1459174448,Wheat Thins Original,NaN,258g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,7832309,http://world.openfoodfacts.org/product/07832309,openfoodfacts-contributors,1403210081,1458995984,Diet Dr Pepper,NaN,NaN,can,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5099353000169,http://world.openfoodfacts.org/product/5099353000169,bcatelin,1385926289,1413659845,Eggs,Eggs,6,Cardbox,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,82592720153,http://world.openfoodfacts.org/product/0082592720153,openfoodfacts-contributors,1389308826,1459174499,Green Machine,NaN,15.2 fl. oz (450 mL),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.532


In [123]:
###verify the nutrition facts add up with the manufacturer's data
### it appears from the site's documentation that 
###http://world.openfoodfacts.org/data/data-fields.txt
###[nutrient]_100gr is nutrients per 100gr or mL
###step 1, grab data for an identifable product

target_list = ['code', 'url', 'product_name', 'energy_100g','carbohydrates_100g', 
    'fat_100g', 'proteins_100g','serving_size', 'vitamin-a_100g', 'vitamin-c_100g', 'calcium_100g', 'iron_100g', 'sodium_100g']

df[target_list].iloc[20]

code                  36632036506                                         
url                   http://world.openfoodfacts.org/product/0036632036506
product_name          Activia light blueberry                             
energy_100g           222                                                 
carbohydrates_100g    9.73                                                
fat_100g              0                                                   
proteins_100g         3.54                                                
serving_size          1 container (113g)                                  
vitamin-a_100g        NaN                                                 
vitamin-c_100g        NaN                                                 
calcium_100g          0.133                                               
iron_100g             NaN                                                 
sodium_100g           0.0575                                              
Name: 20, dtype: object

In [79]:
#step 2: verify the data on the site's own product page
import html5lib
site = 'http://www.activia.us.com/probiotic-yogurt/products/activia-light-blueberry'
blueberry_yougurt = pd.read_html(site)[0].set_index('Nutritional Facts')
blueberry_yougurt

,per serving (113g),Calories from Fat
Nutritional Facts,,
NaN,NaN,% Daily Value*
Calories,60,0
Total Fat,0g,0%
Saturated Fat,0g,0%
Trans Fat,0g,NaN
Cholesterol,<5mg,1%
Sodium,65mg,3%
Potassium,200mg,6%
Total Carbohydrate,11g,4%


In [80]:
### the calroies(enerygy) does not look like it is 220 cals/100gr, but the carb/protein/fat ratio looks about right
### verify the macronutrients per 100gr are equivalent to the food facts db:
print(float(blueberry_yougurt.loc['Protein'][0].strip('g'))/1.13)
print(float(blueberry_yougurt.loc['Total Carbohydrate'][0].strip('g'))/1.13)
print(float(blueberry_yougurt.loc['Total Fat'][0].strip('g'))/1.13)

3.5398230088495577
9.734513274336283
0.0


In [124]:
###the total macronutrients per 100 grams posted in the food facts db roughly matches what is posted on the product's
###own site.  Let's check anoother product, this time at random:
np.random.seed(4)
df[target_list].iloc[np.random.randint(0,len(df))]

code                  27000379097                                         
url                   http://world.openfoodfacts.org/product/0027000379097
product_name          Crushed tomatoes                                    
energy_100g           155                                                 
carbohydrates_100g    7.44                                                
fat_100g              0                                                   
proteins_100g         1.65                                                
serving_size          121 g                                               
vitamin-a_100g        0.000124                                            
vitamin-c_100g        0.00496                                             
calcium_100g          0                                                   
iron_100g             0.00119                                             
sodium_100g           0.19                                                
Name: 1146, dtype: object

In [125]:
site = 'http://www.hunts.com/nutrition-label?upc=2700037909&inline=false'
hunts_tomatoes = pd.read_html(site)
hunts_tomatoes[1]

,0,1,2
0,Nutrition Facts,NaN,NaN
1,Serving Size 1/2 cup (121g),NaN,NaN
2,Servings Per Container about 7,NaN,NaN
3,Amount Per Serving,NaN,NaN
4,Calories 45Calories from Fat 0,NaN,NaN
5,NaN,% Daily Value*,NaN
6,Total Fat 0g,0%,NaN
7,NaN,Saturated Fat 0g,0%
8,NaN,Trans Fat 0g,NaN
9,Cholesterol 0mg,0%,NaN


In [126]:
# let's see how many of our rows have data in these target categories.  
# if we screen out those that are missing target data, how many items will we be left with?

for item in target_list:
    print(item, ":", df[item].count())


code : 2820
url : 2820
product_name : 2356
energy_100g : 1283
carbohydrates_100g : 1292
fat_100g : 1295
proteins_100g : 1284
serving_size : 1362
vitamin-a_100g : 921
vitamin-c_100g : 936
calcium_100g : 948
iron_100g : 952
sodium_100g : 1258


In [127]:
df2 = df[target_list]
df2.head()

,code,url,product_name,energy_100g,carbohydrates_100g,fat_100g,proteins_100g,serving_size,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,sodium_100g
0,5010092093045,http://world.openfoodfacts.org/product/5010092093045,Soft white,1007.0,44.6,2.0,9.000,50g,NaN,NaN,NaN,NaN,0.40000
1,44000030377,http://world.openfoodfacts.org/product/0044000030377,Wheat Thins Original,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7832309,http://world.openfoodfacts.org/product/07832309,Diet Dr Pepper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5099353000169,http://world.openfoodfacts.org/product/5099353000169,Eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82592720153,http://world.openfoodfacts.org/product/0082592720153,Green Machine,244.0,13.8,0.0,0.833,8 fl oz (240 mL),0.000313,0.01,0.00833,0.0003,0.00625


In [128]:
# have a look at some of the rows

df2.iloc[580:605]

,code,url,product_name,energy_100g,carbohydrates_100g,fat_100g,proteins_100g,serving_size,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,sodium_100g
580,12000130274,http://world.openfoodfacts.org/product/0012000130274,NaN,195.0,12.3,0.00,0.0,1 Bottle (20 fl oz) (591 mL),NaN,NaN,NaN,NaN,0.0158
581,810165016415,http://world.openfoodfacts.org/product/0810165016415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
582,856820160048,http://world.openfoodfacts.org/product/0856820160048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583,617237641564,http://world.openfoodfacts.org/product/0617237641564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,8000500026731,http://world.openfoodfacts.org/product/8000500026731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,38000576089,http://world.openfoodfacts.org/product/0038000576089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,8000500205167,http://world.openfoodfacts.org/product/8000500205167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,70896732057,http://world.openfoodfacts.org/product/0070896732057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588,799857655371,http://world.openfoodfacts.org/product/0799857655371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
589,810675000225,http://world.openfoodfacts.org/product/0810675000225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# looks like some of these rows may be poor quality (many NaN fields), example row 581

df2.iloc[581]

code                  810165016415                                        
url                   http://world.openfoodfacts.org/product/0810165016415
product_name          NaN                                                 
energy_100g           NaN                                                 
carbohydrates_100g    NaN                                                 
fat_100g              NaN                                                 
proteins_100g         NaN                                                 
serving_size          NaN                                                 
vitamin-a_100g        NaN                                                 
vitamin-c_100g        NaN                                                 
calcium_100g          NaN                                                 
iron_100g             NaN                                                 
sodium_100g           NaN                                                 
Name: 581, dtype: object

In [130]:
# use notnull to eliminate the rows that inluce NaNs in the fields we wish to study.

clean_df2 = df2[pd.notnull(df['vitamin-a_100g'])]
clean_df2.head()

,code,url,product_name,energy_100g,carbohydrates_100g,fat_100g,proteins_100g,serving_size,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,sodium_100g
4,82592720153,http://world.openfoodfacts.org/product/0082592720153,Green Machine,244.0,13.800,0.00,0.833,8 fl oz (240 mL),0.000313,0.01,0.00833,0.00030,0.00625
6,9800895250,http://world.openfoodfacts.org/product/0009800895250,Nutella,2260.0,56.800,32.40,5.410,2 tbsp (37 g),0.000000,0.00,0.10800,0.00195,0.04050
14,1762949,http://world.openfoodfacts.org/product/01762949,NaN,3350.0,0.000,93.30,NaN,1 tbsp (15 mL),0.000000,0.00,0.00000,0.00000,NaN
22,4154891,http://world.openfoodfacts.org/product/04154891,Almond Milk Unsweetened,52.5,0.833,1.04,0.417,1 cup (240 mL),0.000063,0.00,0.18800,0.00030,0.07500
23,41570054130,http://world.openfoodfacts.org/product/0041570054130,Almond Milk Unsweetened,52.5,0.833,1.04,0.417,1 cup (240 mL),0.000063,0.00,0.18800,0.00030,0.07500


In [131]:
# this new df should be pretty balanced, not so many NaNs left.  let's see.
# note that I generic_name was added back as some of the product_name fields are unhelpful (ie "soft white" - ?)
# it's okay then that this field is a bit underpopulated, we'll use it just to fill in product name when unhelpful

clean_df2.count()

code                  921
url                   921
product_name          884
energy_100g           905
carbohydrates_100g    916
fat_100g              920
proteins_100g         913
serving_size          912
vitamin-a_100g        921
vitamin-c_100g        892
calcium_100g          896
iron_100g             884
sodium_100g           917
dtype: int64

In [89]:
# at this point we are well positioned to be able to analyse calorie count for any subset for our data.
# now what about the other nutrients: does a "basket" provide adequate nutrition, per IOM RDAs?
# let's go back to our original data set and see what nutrient data is available

# use this to search to compare to the nutrients studied by the IOM
# we're going to search on columns that include _100g, many of which are nutrients
# then need to work the formatting, removing _100g (by slicing), and
# replacing the "-" in two places: first character and inside the string

column_list = list(df.columns.values)

products_nutrients_list = []
for column in column_list:
    if re.search('_100g$', column):  # this gives us all the columns that include _100g, many of which are nutrients
        if re.search('^-', column):
            products_nutrients_list.append((column.replace("-", " "))[1:-5])  
        else:
            products_nutrients_list.append((column.replace("-", " "))[0:-5])
            
print(products_nutrients_list)

['energy', 'energy from fat', 'fat', 'saturated fat', 'butyric acid', 'caproic acid', 'caprylic acid', 'capric acid', 'lauric acid', 'myristic acid', 'palmitic acid', 'stearic acid', 'arachidic acid', 'behenic acid', 'lignoceric acid', 'cerotic acid', 'montanic acid', 'melissic acid', 'monounsaturated fat', 'polyunsaturated fat', 'omega 3 fat', 'alpha linolenic acid', 'eicosapentaenoic acid', 'docosahexaenoic acid', 'omega 6 fat', 'linoleic acid', 'arachidonic acid', 'gamma linolenic acid', 'dihomo gamma linolenic acid', 'omega 9 fat', 'oleic acid', 'elaidic acid', 'gondoic acid', 'mead acid', 'erucic acid', 'nervonic acid', 'trans fat', 'cholesterol', 'carbohydrates', 'sugars', 'sucrose', 'glucose', 'fructose', 'lactose', 'maltose', 'maltodextrins', 'starch', 'polyols', 'fiber', 'proteins', 'casein', 'serum proteins', 'nucleotides', 'salt', 'sodium', 'alcohol', 'vitamin a', 'beta carotene', 'vitamin d', 'vitamin e', 'vitamin k', 'vitamin c', 'vitamin b1', 'vitamin b2', 'vitamin pp', '

In [90]:
# pull in data from https://www.consumerlab.com/RDAs/ that shows RDAs (DRIs) from the IOM
# what are the nutrients for which recommendations are available?

nutrients = pd.read_html("https://www.consumerlab.com/RDAs/")[6]
nutrients


,0,1,2,3,4,5
0,Vitamins,B Vitamins,B Vitamins (cont'd),Minerals,Minerals(cont'd),Minerals(cont'd)
1,Vitamin A,Biotin,Riboflavin,Calcium,Iron,Potassium
2,Vitamin C,Choline,Thiamin,Chromium,Magnesium,Selenium
3,Vitamin D,Folate,Vitamin B-6,Copper,Molybdenum,Zinc
4,Vitamin E,Niacin,Vitamin B-12,Fluoride,Manganese,NaN
5,Vitamin K,Pantothenic Acid,NaN,Iodine,Phosphorus,NaN


In [91]:
# we don't need that first row, and we want the NaNs replaced with 0's, so we can screen them out later
# result is a nice, tidy dataframe

nutrients.fillna('x', inplace=True)
nutrients.drop(0, inplace = True)
nutrients

,0,1,2,3,4,5
1,Vitamin A,Biotin,Riboflavin,Calcium,Iron,Potassium
2,Vitamin C,Choline,Thiamin,Chromium,Magnesium,Selenium
3,Vitamin D,Folate,Vitamin B-6,Copper,Molybdenum,Zinc
4,Vitamin E,Niacin,Vitamin B-12,Fluoride,Manganese,x
5,Vitamin K,Pantothenic Acid,x,Iodine,Phosphorus,x


In [92]:
# let's put these in a list

iom_list2 = nutrients.values.flatten().tolist()
iom_list2

['Vitamin A',
 'Biotin',
 'Riboflavin',
 'Calcium',
 'Iron',
 'Potassium',
 'Vitamin C',
 'Choline',
 'Thiamin',
 'Chromium',
 'Magnesium',
 'Selenium',
 'Vitamin D',
 'Folate',
 'Vitamin B-6',
 'Copper',
 'Molybdenum',
 'Zinc',
 'Vitamin E',
 'Niacin',
 'Vitamin B-12',
 'Fluoride',
 'Manganese',
 'x',
 'Vitamin K',
 'Pantothenic Acid',
 'x',
 'Iodine',
 'Phosphorus',
 'x']

In [93]:
# then screen out the x's we substituted in for the NaNs.
# result is a nice tidy list

iom_list3 = [x for x in iom_list2 if len(x) >2]
print(iom_list3)

['Vitamin A', 'Biotin', 'Riboflavin', 'Calcium', 'Iron', 'Potassium', 'Vitamin C', 'Choline', 'Thiamin', 'Chromium', 'Magnesium', 'Selenium', 'Vitamin D', 'Folate', 'Vitamin B-6', 'Copper', 'Molybdenum', 'Zinc', 'Vitamin E', 'Niacin', 'Vitamin B-12', 'Fluoride', 'Manganese', 'Vitamin K', 'Pantothenic Acid', 'Iodine', 'Phosphorus']


In [94]:
# now to format the list in a way that complies with the way our product list appears
# to do this we must turn B-6 into B 6 and lowercase all our strings

iom_nutrients_list = []
for item in iom_list3:
    if re.search('-', item):  
        iom_nutrients_list.append((item.replace("-", "")).lower())
    else:
        iom_nutrients_list.append(item.lower())
            
print(iom_nutrients_list)

['vitamin a', 'biotin', 'riboflavin', 'calcium', 'iron', 'potassium', 'vitamin c', 'choline', 'thiamin', 'chromium', 'magnesium', 'selenium', 'vitamin d', 'folate', 'vitamin b6', 'copper', 'molybdenum', 'zinc', 'vitamin e', 'niacin', 'vitamin b12', 'fluoride', 'manganese', 'vitamin k', 'pantothenic acid', 'iodine', 'phosphorus']


In [95]:
# now that our formatting matches, we can compare 
combined_nutrients_list = []
for item in iom_nutrients_list:
    if item in products_nutrients_list:
        combined_nutrients_list.append(item)
        
print(combined_nutrients_list)

['vitamin a', 'biotin', 'calcium', 'iron', 'potassium', 'vitamin c', 'chromium', 'magnesium', 'selenium', 'vitamin d', 'vitamin b6', 'copper', 'molybdenum', 'zinc', 'vitamin e', 'vitamin b12', 'fluoride', 'manganese', 'vitamin k', 'pantothenic acid', 'iodine', 'phosphorus']


In [96]:
# let's go back and look at all of our columns, and see how many of each are populated
# we'll use this later when determining which characteristics are 'worthwhile' to compare

for column in column_list:
    if re.search('_100g$', column):
        print('{:<50}'.format(column), df[df[column]!= np.nan][column].count())

energy_100g                                        1283
energy-from-fat_100g                               793
fat_100g                                           1295
saturated-fat_100g                                 1124
-butyric-acid_100g                                 0
-caproic-acid_100g                                 0
-caprylic-acid_100g                                0
-capric-acid_100g                                  0
-lauric-acid_100g                                  0
-myristic-acid_100g                                0
-palmitic-acid_100g                                0
-stearic-acid_100g                                 0
-arachidic-acid_100g                               0
-behenic-acid_100g                                 0
-lignoceric-acid_100g                              0
-cerotic-acid_100g                                 0
-montanic-acid_100g                                0
-melissic-acid_100g                                0
monounsaturated-fat_100g           

In [114]:
# and maybe just those that appear in significant (>900) numbers

populated_columns = []
for column in column_list:
    if re.search('_100g$', column):
        if df[df[column]!= np.nan][column].count() > 900:
            print('{:<50}'.format(column), df[df[column]!= np.nan][column].count())
            populated_columns.append(column)
#print(populated_columns)

energy_100g                                        1283
fat_100g                                           1295
saturated-fat_100g                                 1124
trans-fat_100g                                     961
cholesterol_100g                                   962
carbohydrates_100g                                 1292
sugars_100g                                        1197
fiber_100g                                         1028
proteins_100g                                      1284
salt_100g                                          1258
sodium_100g                                        1258
vitamin-a_100g                                     921
vitamin-c_100g                                     936
calcium_100g                                       948
iron_100g                                          952
nutrition-score-fr_100g                            1110
nutrition-score-uk_100g                            1110
calories_100g                                      127

In [98]:
# let's bring that combined nutrients list back and compare it to our numbers above

print(combined_nutrients_list)

['vitamin a', 'biotin', 'calcium', 'iron', 'potassium', 'vitamin c', 'chromium', 'magnesium', 'selenium', 'vitamin d', 'vitamin b6', 'copper', 'molybdenum', 'zinc', 'vitamin e', 'vitamin b12', 'fluoride', 'manganese', 'vitamin k', 'pantothenic acid', 'iodine', 'phosphorus']


In [99]:
# now let's read in the actual recommended values from the website.
# this will need some grooming 

nutrients2 = pd.read_html("https://www.consumerlab.com/RDAs/")[7]
nutrients2.fillna('x', inplace=True)
nutrients2

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Age,Vitamin ASee Report,Folate(Vitamin B-9)See Report,Vitamin CSee Report,Vitamin DSee Report,CalciumSee Report,IronSee Report,x,x,x,x,x,x
1,RDA1,Upper Limit2,RDA3,Upper Limit4,RDA,Upper Limit,RDA5,Upper Limit,RDA,Upper Limit,RDA,Upper Limit,x
2,1 - 3,"1,000 IU","2,000 IU",150 mcg,300 mcg,15 mg,400 mg,600 IU,"2,500 IU",700mg,"2,500 mg",7 mg,40 mg
3,4 - 8,"1,300 IU","3,000 IU",200 mcg,400 mcg,25 mg,650 mg,600 IU,"3,000 IU","1,000 mg","2,500 mg",10 mg,40 mg
4,9 - 13,"2,000 IU","5,666 IU",300 mcg,600 mcg,45 mg,"1,200 mg",600 IU,"4,000 IU","1,300 mg","3,000 mg",8 mg,40 mg
5,14 - 18,"1,000 IU","9,333 IU",400 mcg,800 mcg,75 mg (m) 65 mg (f) 80 mg (preg) 115 mg (lact),"1,800 mg",600 IU,"4,000 IU","1,300 mg","3,000 mg",11 mg (m) 15 mg (f) 27 mg (preg)10 mg (lact),45 mg
6,Adult,"3,000 IU (m)2,300 IU (f)","10,000 IU",400 mcg 600 mcg (preg)/ 500 mcg (lact),"1,000 mcg",90 (m) 75 mg (f) 85 mg (preg) 120 (lact),"2,000 mg",600 IU (51- 70 years) 800 IU (71+ years),"4,000 IU","1,000 mg (to 50 years) 1,200 mg (51+ years)","2,500 mg (to 50 years) 2,000 mg (51+ years)",8 mg (m) 18 mg (f 19 to 50 years) 8 mg (f 51+ years) 27 mg (preg) 9 mg (lact),45 mg
7,DV:,x,x,x,x,x,x,x,x,x,x,x,x
8,0 - 12 mo.,500 mcg(RAE)6,80 mcg(DFE)7,50 mg,10 mcg5 (400 IU),260 mg,11 mg,x,x,x,x,x,x
9,1 -3,300 mcg(RAE)6,150 mcg(DFE)7,15 mg,15 mcg5 (600 IU),700 mg,7 mg,x,x,x,x,x,x


In [100]:
# we're going to measure intake for an Adult.  here's the data we need

nutrients2.iloc[6]

0     Adult                                                                          
1     3,000 IU (m)2,300 IU (f)                                                       
2     10,000 IU                                                                      
3     400 mcg 600 mcg (preg)/ 500 mcg (lact)                                         
4     1,000 mcg                                                                      
5     90 (m) 75 mg (f) 85 mg (preg) 120 (lact)                                       
6     2,000 mg                                                                       
7     600 IU (51- 70 years) 800 IU (71+ years)                                       
8     4,000 IU                                                                       
9     1,000 mg  (to 50 years) 1,200 mg (51+ years)                                   
10    2,500 mg (to 50 years) 2,000 mg (51+ years)                                    
11    8 mg (m)  18 mg (f 19 to 50 years)  8 mg (f 51+ 

In [101]:
# we need to match our values above with the categories, from rows 1 & 2 in our dataframe, which need tidying
# first put the data from one of the rows in a list.  the NaNs (x's) are all nicely colledted at the end of the list,
# so a slice will serve to eliminate these

col0 = (nutrients2.iloc[0].values.flatten().tolist())[1:7]
print(col0)

['Vitamin ASee Report', 'Folate(Vitamin B-9)See Report', 'Vitamin CSee Report', 'Vitamin DSee Report', 'CalciumSee Report', 'IronSee Report']


In [102]:
# we don't need the len(col)>2 here really as we've already eliminated NaNs but in case one sneaks through
# strip the "See Report" piece from the end of the nutrient title, using RegEx.

col1 = []
for col in col0:
    if len(col) >2:
        col1.append(re.split("See Report", col)[0])
print(col1)

['Vitamin A', 'Folate(Vitamin B-9)', 'Vitamin C', 'Vitamin D', 'Calcium', 'Iron']


In [103]:
# now each of the vitamins above has a value for 'RDA' and a value for 'Upper limit'
# have just typed these in because it seemed easier.  so lazy!

col2 = ['RDA', 'Upper Limit']

In [104]:
# our new column headers will include two different versions of each nutrient.  let's build the list.
# on the first run-through, error suggested there were too few column headers.  
# this is because we've chosen to ignore that weird first column rather than dealing with it earlier. 
# so now to deal with this, we'll just add a blank column to our header list

cols = [" "]
for x in col1:
    for y in col2:
        cols.append(x + " " + y)

print(cols)

[' ', 'Vitamin A RDA', 'Vitamin A Upper Limit', 'Folate(Vitamin B-9) RDA', 'Folate(Vitamin B-9) Upper Limit', 'Vitamin C RDA', 'Vitamin C Upper Limit', 'Vitamin D RDA', 'Vitamin D Upper Limit', 'Calcium RDA', 'Calcium Upper Limit', 'Iron RDA', 'Iron Upper Limit']


In [105]:
# BOOM!

nutrients2.columns = cols
nutrients2.iloc[4:7]

,,Vitamin A RDA,Vitamin A Upper Limit,Folate(Vitamin B-9) RDA,Folate(Vitamin B-9) Upper Limit,Vitamin C RDA,Vitamin C Upper Limit,Vitamin D RDA,Vitamin D Upper Limit,Calcium RDA,Calcium Upper Limit,Iron RDA,Iron Upper Limit
4,9 - 13,"2,000 IU","5,666 IU",300 mcg,600 mcg,45 mg,"1,200 mg",600 IU,"4,000 IU","1,300 mg","3,000 mg",8 mg,40 mg
5,14 - 18,"1,000 IU","9,333 IU",400 mcg,800 mcg,75 mg (m) 65 mg (f) 80 mg (preg) 115 mg (lact),"1,800 mg",600 IU,"4,000 IU","1,300 mg","3,000 mg",11 mg (m) 15 mg (f) 27 mg (preg)10 mg (lact),45 mg
6,Adult,"3,000 IU (m)2,300 IU (f)","10,000 IU",400 mcg 600 mcg (preg)/ 500 mcg (lact),"1,000 mcg",90 (m) 75 mg (f) 85 mg (preg) 120 (lact),"2,000 mg",600 IU (51- 70 years) 800 IU (71+ years),"4,000 IU","1,000 mg (to 50 years) 1,200 mg (51+ years)","2,500 mg (to 50 years) 2,000 mg (51+ years)",8 mg (m) 18 mg (f 19 to 50 years) 8 mg (f 51+ years) 27 mg (preg) 9 mg (lact),45 mg


In [106]:
# on second glance those values are very ugly and will need some grooming also.

adult_rda = list(nutrients2.iloc[6].values[1:])
print(adult_rda)

['3,000 IU (m)2,300 IU (f)', '10,000 IU', '400 mcg 600 mcg (preg)/ 500 mcg (lact)', '1,000 mcg', '90 (m) 75 mg (f) 85 mg (preg) 120 (lact)', '2,000 mg', '600 IU (51- 70 years) 800 IU (71+ years)', '4,000 IU', '1,000 mg  (to 50 years) 1,200 mg (51+ years)', '2,500 mg (to 50 years) 2,000 mg (51+ years)', '8 mg (m)  18 mg (f 19 to 50 years)  8 mg (f 51+ years) 27 mg (preg) 9 mg (lact)', '45 mg']


In [107]:
# use RegEx to extract the RDA values and units into a list.  not sure if we need units at this moment, can come back
# we can see the adult male values are listed first, so we'll use indexing to extract the value we need

rda_vals = []
for item in adult_rda:
    nums_list = re.findall('[0-9,]+', item)
    print(nums_list)
    units_list = re.findall('[a-zA-Z]+', item)
    print(units_list)
    rda_vals.append(nums_list[0] + " " + units_list[0])

print(rda_vals)    


['3,000', '2,300']
['IU', 'm', 'IU', 'f']
['10,000']
['IU']
['400', '600', '500']
['mcg', 'mcg', 'preg', 'mcg', 'lact']
['1,000']
['mcg']
['90', '75', '85', '120']
['m', 'mg', 'f', 'mg', 'preg', 'lact']
['2,000']
['mg']
['600', '51', '70', '800', '71']
['IU', 'years', 'IU', 'years']
['4,000']
['IU']
['1,000', '50', '1,200', '51']
['mg', 'to', 'years', 'mg', 'years']
['2,500', '50', '2,000', '51']
['mg', 'to', 'years', 'mg', 'years']
['8', '18', '19', '50', '8', '51', '27', '9']
['mg', 'm', 'mg', 'f', 'to', 'years', 'mg', 'f', 'years', 'mg', 'preg', 'mg', 'lact']
['45']
['mg']
['3,000 IU', '10,000 IU', '400 mcg', '1,000 mcg', '90 m', '2,000 mg', '600 IU', '4,000 IU', '1,000 mg', '2,500 mg', '8 mg', '45 mg']


In [234]:
# lets put these values & units in a dictionary with the column headers, for later use

rda_dict = dict(zip(cols[1:], rda_vals))
print(rda_dict)

{'Calcium RDA': '1,000 mg', 'Vitamin D RDA': '600 IU', 'Folate(Vitamin B-9) RDA': '400 mcg', 'Calcium Upper Limit': '2,500 mg', 'Iron RDA': '8 mg', 'Vitamin A Upper Limit': '10,000 IU', 'Vitamin A RDA': '3,000 IU', 'Folate(Vitamin B-9) Upper Limit': '1,000 mcg', 'Iron Upper Limit': '45 mg', 'Vitamin D Upper Limit': '4,000 IU', 'Vitamin C Upper Limit': '2,000 mg', 'Vitamin C RDA': '90 m'}


In [250]:
# coming back to this, we only need RDAs, not Upper Limits, and we need to standardize the format of those keys
# create a new tidier dictionary for RDAs

rda_dict_tidy = {}
for key in rda_dict:
    if re.search('RDA', key):
        rda_dict_tidy[key.lower()[:-4]] = rda_dict[key]

print(rda_dict_tidy)

{'calcium': '1,000 mg', 'vitamin d': '600 IU', 'vitamin c': '90 m', 'vitamin a': '3,000 IU', 'folate(vitamin b-9)': '400 mcg', 'iron': '8 mg'}


In [235]:
# we might actually be able to solve this problem by using means from the preliminary data set.
# let's have a look

df.mean()


Unnamed: 0                                 1.409500e+03
code                                       5.784618e+12
created_t                                  1.430248e+09
last_modified_t                            1.443026e+09
cities                                    NaN          
no_nutriments                             NaN          
additives_n                                2.174249e+00
ingredients_from_palm_oil_n                8.583691e-04
ingredients_from_palm_oil                 NaN          
ingredients_that_may_be_from_palm_oil_n    2.145923e-02
ingredients_that_may_be_from_palm_oil     NaN          
energy_100g                                1.179256e+03
energy-from-fat_100g                       5.814481e+02
fat_100g                                   1.358967e+01
saturated-fat_100g                         5.417324e+00
-butyric-acid_100g                        NaN          
-caproic-acid_100g                        NaN          
-caprylic-acid_100g                       NaN   

In [236]:
df['vitamin-a_100g'].mean()

0.00012034548317046676

In [237]:
df['calories_100g'].mean()

289.772216535433

In [238]:
# so we started with 2040g of food for per day, but the calories implied by this quantity were far too high (>6000)
# let's see instead using our calorie column, how many grams of food are allowed (answer is 862.7g)

constant = 2500/df['calories_100g'].mean()
print(constant)


8.627466186684266


In [257]:
# now let's find out if a person consume that amount of food each day, what nutrient content might result
# find the average nutrient content within our set (per 100g) and multiply it by our constant
# this mean_vals list accounts for a full day's consumption
# it also appears that the data set measures nutrients in grams, so we need to convert to micrograms

daily_amts = []
for item in populated_columns:
    print('{:<50}'.format(item), df[item].mean())
    daily_amts.append("%.2f" % (df[item].mean()*constant*1000))
    
print(daily_amts)

energy_100g                                        1179.2562665627433
fat_100g                                           13.589665637065645
saturated-fat_100g                                 5.417323843416372
trans-fat_100g                                     0.07857775234131113
cholesterol_100g                                   0.018391049896049904
carbohydrates_100g                                 35.559534055727525
sugars_100g                                        16.587055973266494
fiber_100g                                         3.0853229571984473
proteins_100g                                      6.568091121495342
salt_100g                                          1.2329645405405394
sodium_100g                                        0.48521631947974025
vitamin-a_100g                                     0.00012034548317046676
vitamin-c_100g                                     0.005969212606837604
calcium_100g                                       0.09144454641350203
iron_100g  

In [258]:
tidy_pop_columns = []
for item in populated_columns:
    if re.search('-', item):  
        tidy_pop_columns.append((item.replace("-", " ")).lower()[:-5])
    else:
        tidy_pop_columns.append(item.lower()[:-5])

print(tidy_pop_columns)

['energy', 'fat', 'saturated fat', 'trans fat', 'cholesterol', 'carbohydrates', 'sugars', 'fiber', 'proteins', 'salt', 'sodium', 'vitamin a', 'vitamin c', 'calcium', 'iron', 'nutrition score fr', 'nutrition score uk', 'calories']


In [259]:
# let's look at our two dictionaries (the average daily nutrient content and the RDA) side-by-side
# start with our daily average

daily_amts_dict = dict(zip(tidy_pop_columns, daily_amts))
print(daily_amts_dict)

{'proteins': '56665.98', 'calories': '2500000.00', 'trans fat': '677.93', 'vitamin c': '51.50', 'fiber': '26618.52', 'salt': '10637.36', 'nutrition score fr': '76916.58', 'sodium': '4186.19', 'calcium': '788.93', 'fat': '117244.38', 'saturated fat': '46737.78', 'energy': '10173993.57', 'carbohydrates': '306788.68', 'iron': '20.52', 'cholesterol': '158.67', 'vitamin a': '1.04', 'nutrition score uk': '75742.94', 'sugars': '143104.26'}


In [260]:
# then look at the RDAs

print(rda_dict_tidy)

{'calcium': '1,000 mg', 'vitamin d': '600 IU', 'vitamin c': '90 m', 'vitamin a': '3,000 IU', 'folate(vitamin b-9)': '400 mcg', 'iron': '8 mg'}


In [261]:
# we only need RDAs, not Upper Limits (will never get there), let's compare them to our means

for item in daily_amts_dict.keys():
    if item in rda_dict_tidy.keys():
        print(item, "consumed", daily_amts_dict[item], "recommended", rda_dict_tidy[item])

vitamin c consumed 51.50 recommended 90 m
calcium consumed 788.93 recommended 1,000 mg
iron consumed 20.52 recommended 8 mg
vitamin a consumed 1.04 recommended 3,000 IU
